# Part_1_5_Words_Corpora_Text_Normalization
In Natural Language Processing (NLP), key techniques such as tokenization, stemming, and sentence segmentation are fundamental for transforming raw text into a structured format that can be effectively analyzed by language models. Byte-Pair Encoding (BPE) tokenization helps handle rare words and out-of-vocabulary terms by breaking text into subword units. Stemming, using algorithms like the Porter Stemmer, reduces words to their root forms, ensuring consistency across the text for better analysis. Sentence segmentation is crucial for dividing text into meaningful sentences, allowing for more accurate processing in downstream tasks. These techniques play a vital role in preparing text for various NLP tasks, ensuring that language data is in a normalized and analyzable state.

### **Objectives:**
By the end of this notebook, Parham will have a thorough understanding of tokenization and its importance in NLP, specifically learning how to implement **Byte-Pair Encoding (BPE)** to handle words and subwords. He will explore the **Porter Stemmer**, gaining insight into how it reduces words to their base forms and why this is essential for text normalization. Additionally, Parham will learn to apply **Sentence Segmentation** to split text into meaningful sentences for deeper analysis. Through hands-on coding exercises, he will gain practical experience using these techniques, utilizing Python libraries like `NLTK` and `SpaCy` to prepare text for NLP tasks.

**Table of Contents:** 
1. Import Libraries
2. Tokenization Techniques
3. Text Normalization: Porter Stemmer
4. Sentence Segmentation
5. Closing Thoughts

## 1. Import Libraries

In [4]:
import re
import os
import sys
from loguru import logger
import nltk
import numpy as np
from collections import Counter, defaultdict

## 2. Tokenization Techniques
Byte-pair encoding was first introduced in 1994 as a simple data compression technique by iteratively replacing the most frequent pair of bytes in a sequence with a single, unused byte.
Imagine Parham is reading a really big book, but some of the words are really long or tricky, and he might not know all of them. To make things easier, Parham can break the long words into smaller, simpler parts or pieces. This way, he can still understand the book without needing to know every single big word.

Byte-Pair Encoding (BPE) is a popular technique used in natural language processing for tokenizing text into subword units. The idea is to break down words into smaller, more frequent pieces, allowing models to efficiently handle rare or unknown words. BPE is especially useful in scenarios where the vocabulary is limited but the text contains a large variety of words, including compound or out-of-vocabulary words.

The algorithm works by:
- Starting with a sequence of individual characters.
- Finding the most frequent pair of consecutive characters (or subwords).
- Merging this pair into a single token.
- Repeating the process until a predefined number of merges or a desired vocabulary size is reached.

This process allows models to represent both common words and subwords effectively, making it easier to process any text.

In the cell below Parham will an example of BEP tokenization.

In [5]:
def get_stats(vocab):
    """
    Given a vocabulary (dictionary mapping words to frequency counts), returns a 
    dictionary of tuples representing the frequency count of pairs of characters 
    in the vocabulary.
    """
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    """
    Given a pair of characters and a vocabulary, returns a new vocabulary with the 
    pair of characters merged together wherever they appear.
    """
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_vocab(data):
    """
    Given a list of strings, returns a dictionary of words mapping to their frequency 
    count in the data.
    """
    vocab = defaultdict(int)
    for line in data:
        for word in line.split():
            vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

def byte_pair_encoding(data, n):
    """
    Given a list of strings and an integer n, returns a list of n merged pairs
    of characters found in the vocabulary of the input data.
    """
    vocab = get_vocab(data)
    for i in range(n):
        pairs = get_stats(vocab)
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)
        print(f"\nAfter merge {i + 1}:")
        print(f"Best Pair: {best}")
        print(f"Updated Vocabulary: {vocab}")
    return vocab

# Example usage:
corpus = '''Tokenization is the process of breaking down 
a sequence of text into smaller units called tokens,
which can be words, phrases, or even individual characters.
Tokenization is often the first step in natural languages processing tasks 
such as text classification, named entity recognition, and sentiment analysis.
The resulting tokens are typically used as input to further processing steps,
such as vectorization, where the tokens are converted
into numerical representations for machine learning models to use.'''
data = corpus.split(' ')

n = 230
bpe_pairs = byte_pair_encoding(data, n)



After merge 1:
Best Pair: ('s', '</w>')
Updated Vocabulary: {'T o k e n i z a t i o n </w>': 2, 'i s</w>': 2, 't h e </w>': 3, 'p r o c e s s</w>': 1, 'o f </w>': 2, 'b r e a k i n g </w>': 1, 'd o w n </w>': 1, 'a </w>': 1, 's e q u e n c e </w>': 1, 't e x t </w>': 2, 'i n t o </w>': 2, 's m a l l e r </w>': 1, 'u n i t s</w>': 1, 'c a l l e d </w>': 1, 't o k e n s , </w>': 1, 'w h i c h </w>': 1, 'c a n </w>': 1, 'b e </w>': 1, 'w o r d s , </w>': 1, 'p h r a s e s , </w>': 1, 'o r </w>': 1, 'e v e n </w>': 1, 'i n d i v i d u a l </w>': 1, 'c h a r a c t e r s . </w>': 1, 'o f t e n </w>': 1, 'f i r s t </w>': 1, 's t e p </w>': 1, 'i n </w>': 1, 'n a t u r a l </w>': 1, 'l a n g u a g e s</w>': 1, 'p r o c e s s i n g </w>': 2, 't a s k s</w>': 1, 's u c h </w>': 2, 'a s</w>': 3, 'c l a s s i f i c a t i o n , </w>': 1, 'n a m e d </w>': 1, 'e n t i t y </w>': 1, 'r e c o g n i t i o n , </w>': 1, 'a n d </w>': 1, 's e n t i m e n t </w>': 1, 'a n a l y s i s . </w>': 1, 'T h e 

In [6]:
import sentencepiece as spm

# Train a SentencePiece model
spm.SentencePieceTrainer.train(input="input.txt", model_prefix='m', vocab_size=100)

# Load the model
sp = spm.SentencePieceProcessor(model_file='m.model')

# Encode a sample text
text = "Hello, how are you?"
encoded = sp.encode(corpus, out_type=str)
print("Encoded:", encoded)

# Decode back to text
decoded = sp.decode(encoded)
print("Decoded:", decoded)


Encoded: ['▁', 'T', 'o', 'k', 'e', 'n', 'i', 'z', 'ation', '▁', 'i', 's', '▁the', '▁', 'p', 'r', 'o', 'c', 'e', 's', 's', '▁', 'o', 'f', '▁', 'b', 're', 'a', 'k', 'ing', '▁', 'd', 'o', 'w', 'n', '▁', 'a', '▁', 's', 'e', 'q', 'u', 'e', 'n', 'c', 'e', '▁', 'o', 'f', '▁', 't', 'e', 'x', 't', '▁in', 't', 'o', '▁', 's', 'm', 'a', 'l', 'l', 'e', 'r', '▁', 'u', 'n', 'i', 't', 's', '▁', 'c', 'a', 'l', 'l', 'e', 'd', '▁to', 'k', 'e', 'n', 's', ',', '▁wh', 'i', 'ch', '▁', 'can', '▁be', '▁', 'w', 'or', 'd', 's', ',', '▁', 'p', 'h', 'r', 'a', 's', 'e', 's', ',', '▁', 'or', '▁', 'e', 'v', 'e', 'n', '▁in', 'd', 'i', 'v', 'i', 'd', 'u', 'a', 'l', '▁', 'ch', 'a', 'r', 'a', 'c', 't', 'e', 'r', 's', '.', '▁', 'T', 'o', 'k', 'e', 'n', 'i', 'z', 'ation', '▁', 'i', 's', '▁', 'o', 'f', 't', 'e', 'n', '▁the', '▁', 'f', 'i', 'r', 's', 't', '▁', 's', 't', 'e', 'p', '▁in', '▁', 'n', 'at', 'u', 'r', 'a', 'l', '▁', 'l', 'a', 'n', 'g', 'u', 'age', 's', '▁processing', '▁', 't', 'a', 's', 'k', 's', '▁', 's', 'u', 'c

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: a.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 100
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_privacy

## 3. Text Normalization: Porter Stemmer

## 4. Sentence Segmentation

## 5. Closing Thoughts